In [ ]:
import string
from copy import deepcopy as copy

from IPython.display import Image, clear_output, display
from Libs.Beholder import *
from PIL import Image as pilImage
from tqdm.notebook import tqdm

In [ ]:
a = Beholder(videoFrameGenerator=pullPhoneScreen)

a.addLayerModifer(
    Beholder_Layer_Chopper_Grayscale(name="gray", from_layer="image", enabled=True)
)

a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="gray_letters",
        from_layer="gray",
        x=304,
        y=1632,
        w=850,
        h=860,
        enabled=True,
    )
)

In [ ]:
from pathlib import Path

import requests

w = Path("words.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://raw.githubusercontent.com/MariaLuo826/english_words/master/words_alpha.txt"
        ).content
    )
words = [l.upper() for l in w.read_text().splitlines()]
len(words)

In [ ]:
# Top words
w = Path("1ktopwords.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://gist.githubusercontent.com/deekayen/4148741/raw/98d35708fa344717d8eee15d11987de6c8e26d7d/1-1000.txt"
        ).content
    )
topwords = [l.upper() for l in w.read_text().splitlines()]
len(topwords)

In [ ]:
# wiki words
w = Path("wikiwords.lst")
if not w.exists():
    w.write_bytes(
        requests.get(
            "https://gist.githubusercontent.com/h3xx/1976236/raw/bbabb412261386673eff521dddbe1dc815373b1d/wiki-100k.txt"
        ).content
    )
wikiwords = list(sorted(list(set([l.upper() for l in w.read_text().splitlines()]))))
len(wikiwords)

In [ ]:
# h3x words
w = Path("1ktopwords.lst")
if not w.exists():
    w.write_bytes(requests.get("").content)
topwords = [l.upper() for l in w.read_text().splitlines()]
len(topwords)

In [ ]:
a.matchers = {}
for l in string.ascii_uppercase:
    p = Path(f"./wordtrip_templates/{l}.png")
    if p.exists():
        print(f"Loading {l}")
        a.addMatcher(
            Beholder_Image_Matcher(
                name=l,
                layer="gray_letters",
                filename=str(p),
                threshhold=0.9,
            )
        )

for l in ["claim_backpack"]:
    p = Path(f"./wordtrip_templates/{l}.png")
    if p.exists():
        print(f"Loading {l}")
        a.addMatcher(
            Beholder_Image_Matcher(
                name=l,
                layer="gray",
                filename=str(p),
                threshhold=0.9,
            )
        )
for p in Path("./wordtrip_templates").glob("close_*.png"):
    print(p)
    a.addMatcher(
        Beholder_Image_Matcher(
            name=p.stem,
            layer="gray",
            filename=str(p),
            threshhold=0.9,
        )
    )
"""
for l in ["white_x"]:
    print(f"Loading masked {l}")
    p = Path(f"./wordtrip_templates/{l}.png")
    a.addMatcher(
        Beholder_Image_Matcher(
            name=l,
            layer="gray",
            filename=str(p),
            mask_filename=f"./wordtrip_templates/{l}_mask.png",
            threshhold=0.9,
        )
    )"""

In [ ]:
def wordWorks(word, found_letters):
    found_letters_t = copy(found_letters)
    for letter in word:
        if letter in found_letters_t:
            found_letters_t.remove(letter)
        else:
            return False
    return True

In [ ]:
def generateMoves(myword, matches):
    mymatches = copy(matches)
    moves = []
    for letter in myword:
        if letter in mymatches:
            if len(mymatches[letter]) > 0:
                m = mymatches[letter].pop(0)
                moves.append(m.center)

    return moves

In [156]:
def gsColor(layer, x, y, correct=None):
    color_data = layer.data[y][x]
    if correct is None:
        return color_data
    for index, element in enumerate(color_data):
        if abs(color_data[index] - correct[index]) > 10:
            print("faile on ", abs(color_data[index] - correct[index]))
            return False
    return True

In [162]:
def get_color_distance_fast(color_1, color_2):
    dist = abs(color_1[0] - color_2[0])
    dist += abs(color_1[1] - color_2[1])
    dist += abs(color_1[2] - color_2[2])
    return dist

In [169]:
get_color_distance_fast(correct, gsColor(a.layers["image"], 58, 58))

0

In [171]:
get_color_distance_fast(correct, gsColor(a.layers["image"], 1190, 1588))

33

In [172]:
get_color_distance_fast(correct, gsColor(a.layers["image"], 700, 1588))

93

In [ ]:
min_len = 3
max_len = 5
eq_len = None

# Manual matcher setup 

In [ ]:
matches = a.findMatches()
print(matches)
found_letters = []
for l in matches:
    for _ in range(0, len(matches[l])):
        found_letters.append(l)
print(found_letters)

matching_words = []
for word in tqdm(words):
    if len(word) < min_len or len(word) > max_len:
        continue
    if wordWorks(word, found_letters):
        matching_words.append(word)
matching_words = list(sorted(matching_words, key=len, reverse=True))
olen = 0
for word in matching_words:
    if len(word) != olen:
        print()
        olen = len(word)
    print(word, end=":")
    print(generateMoves(word, matches))

In [ ]:
correct = [99, 69, 8]

print(gsColor(a.layers["image"], 58, 58))

In [ ]:
raise Exception("Stop")

# Monkey Test

In [158]:
import socket
import subprocess
import sys


class Monkey:
    def __init__(self, create_monkey=True):
        self.create_monkey = create_monkey
        self.port = 9905
        self.p = None
        self.s = None

    def __enter__(self):
        if self.p is not None:
            self.p.kill()
        adb(["forward", "--remove-all"])
        command = ["adb", "shell", "monkey", f"--port {self.port}"]
        self.p = shell(command, close_fds=True, returns_POpen=True)
        adb(f"forward tcp:{self.port} tcp:{self.port}")
        time.sleep(0.75)
        self.connect()
        return self

    def connect(self):
        HOST = "127.0.0.1"
        PORT = self.port

        if self.s is not None:
            self.s.close()
        self.s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.s.connect((HOST, PORT))

    def raw(self, s):
        try:
            self.s.sendall(f"{s}\r\n".encode("ascii"))
            # return self.s.recv(10)
        except Exception as e:
            print("raw", e)
            self.connect()
            return self.raw(s)

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if self.create_monkey:
            self.p.kill()
            self.p = None
            if self.s is not None:
                self.s.close()
                self.s = None
            print("Cleaning up")
            shell(["adb", "forward", "--remove-all"])

In [ ]:
with Monkey() as monkey:
    while True:
        matches = a.findMatches()
        clicked = False

        closeButtones = list(
            filter(
                None, [name if name.startswith("close_") else None for name in matches]
            )
        )
        if len(closeButtones) > 0:
            m = matches[closeButtones[0]][0]
            print(f"clicking {m.matcher.name}")
            c = m.center
            cmd = f"touch down {c[0]} {c[1]}"
            monkey.raw(cmd)
            for _ in range(0, 10):
                for i in range(-2, 2):
                    for y in range(-2, 2):
                        cmd = f"touch move {c[0]+i} {c[1]+y}"
                        monkey.raw(cmd)

            cmd = f"touch up {c[0]} {c[1]}"
            monkey.raw(cmd)
            continue
        if "claim_backpack" in matches:
            m = matches["claim_backpack"][0]
            print(f"clicking {m.matcher.name}")
            c = m.center
            adb(f"shell input tap {c[0]} {c[1]}")
            continue
        found_letters = []
        for l in matches:
            for _ in range(0, len(matches[l])):
                found_letters.append(l)
        print(found_letters)
        if len(found_letters) == 0:
            sleep(3)
            continue
        matching_words = []
        for word in tqdm(words):
            if len(word) < min_len or len(word) > max_len:
                continue
            if wordWorks(word, found_letters):
                matching_words.append(word)

        plen = 0
        matching_words = list(sorted(matching_words, key=len, reverse=True))
        print(matching_words)
        for word in matching_words:
            a.readNextImage()
            if (
                get_color_distance_fast(
                    [74, 57, 58], gsColor(a.layers["image"], 682, 64)
                )
                < 40
            ):
                print(word)
                x = y = 0
                path = generateMoves(word, matches)
                print(path)
                isDown = False
                for x, y in path:
                    if not isDown:
                        cmd = f"touch down {int(x)} {int(y)}"
                        monkey.raw(cmd)
                        isDown = True
                        time.sleep(0.5)
                        continue
                    cmd = f"touch move {int(x)} {int(y)}"
                    monkey.raw(cmd)
                    time.sleep(0.2)
                cmd = f"touch up {int(x)} {int(y)}"
                monkey.raw(cmd)
                sleep(0.6)
            else:
                print("finished i think")
                break
        sleep(3)

  bash arg: --port
  bash arg: 9905


args: [--port, 9905]
 arg: "--port"
 arg: "9905"
arg="--port" mCurArgData="null" mNextArg=1 argwas="--port" nextarg="9905"
data="9905"


    // activityResuming(in.playsimple.wordtrip)
AddMatch close_0 (723, 44)
clicking close_0
AddMatch H (471, 2205)
AddMatch O (810, 2193)
AddMatch R (357, 1861)
AddMatch T (931, 1850)
AddMatch W (612, 1647)
['H', 'O', 'R', 'T', 'W']


  0%|          | 0/370105 [00:00<?, ?it/s]

['ROWTH', 'THROW', 'WHORT', 'WORTH', 'WROTH', 'HORT', 'ORTH', 'ROWT', 'THOR', 'THOW', 'THRO', 'TROW', 'WORT', 'HOR', 'HOT', 'HOW', 'HWT', 'ORT', 'OWT', 'RHO', 'ROT', 'ROW', 'RTW', 'THO', 'TOR', 'TOW', 'TWO', 'WHO', 'WHR', 'WOT', 'WRO']
finished i think
[]
[]
[]
[]
[]
[]
AddMatch H (471, 2205)
AddMatch O (810, 2193)
AddMatch R (357, 1861)
AddMatch T (931, 1850)
AddMatch W (612, 1647)
['H', 'O', 'R', 'T', 'W']


...

['TAWER', 'WATER', 'WREAT', 'ARET', 'AREW', 'ARTE', 'ERAT', 'RATE', 'TARE', 'TEAR', 'TERA', 'TEWA', 'TWAE', 'WAER', 'WARE', 'WART', 'WEAR', 'WERT', 'WETA', 'AER', 'AET', 'ARE', 'ART', 'ATE', 'AWE', 'EAR', 'EAT', 'ERA', 'ETA', 'RAT', 'RAW', 'REA', 'RET', 'REW', 'RTE', 'RTW', 'TAE', 'TAR', 'TAW', 'TEA', 'TER', 'TEW', 'TRA', 'TWA', 'WAE', 'WAR', 'WAT', 'WEA', 'WER', 'WET']
TAWER
[(466, 2188), (813, 2179), (325, 1856), (934, 1863), (645, 1652)]
WATER
[(325, 1856), (813, 2179), (466, 2188), (934, 1863), (645, 1652)]
WREAT
[(325, 1856), (645, 1652), (934, 1863), (813, 2179), (466, 2188)]
ARET
[(813, 2179), (645, 1652), (934, 1863), (466, 2188)]
AREW
[(813, 2179), (645, 1652), (934, 1863), (325, 1856)]
ARTE
[(813, 2179), (645, 1652), (466, 2188), (934, 1863)]
ERAT
[(934, 1863), (645, 1652), (813, 2179), (466, 2188)]
RATE
[(645, 1652), (813, 2179), (466, 2188), (934, 1863)]
TARE
[(466, 2188), (813, 2179), (645, 1652), (934, 1863)]
TEAR
[(466, 2188), (934, 1863), (813, 2179), (645, 1652)]
TERA


  0%|          | 0/370105 [00:00<?, ?it/s]

['FORUM', 'FORM', 'FOUR', 'FROM', 'FOR', 'FOU', 'FRO', 'FUM', 'FUR', 'MFR', 'MOR', 'MOU', 'MRU', 'ORF', 'OUF', 'OUR', 'ROM', 'RUM', 'UFO', 'URF', 'URO']
finished i think
[]
[]
AddMatch F (820, 2195)
AddMatch M (453, 2200)
AddMatch O (633, 1646)
AddMatch R (357, 1861)
AddMatch U (930, 1861)
['F', 'M', 'O', 'R', 'U']


  0%|          | 0/370105 [00:00<?, ?it/s]

['FORUM', 'FORM', 'FOUR', 'FROM', 'FOR', 'FOU', 'FRO', 'FUM', 'FUR', 'MFR', 'MOR', 'MOU', 'MRU', 'ORF', 'OUF', 'OUR', 'ROM', 'RUM', 'UFO', 'URF', 'URO']
FORUM
[(820, 2195), (633, 1646), (357, 1861), (930, 1861), (453, 2200)]
FORM
[(820, 2195), (633, 1646), (357, 1861), (453, 2200)]


# Research


In [ ]:
raise Exception("Stop")

In [ ]:
channels = cv2.split(a.matchers["white_x"].data)
zero_channel = np.zeros_like(channels[0])

In [ ]:
mask = np.array(channels[3])

In [191]:
matches = a.findMatches()
matches

AddMatch D (473, 2196)
AddMatch F (644, 1648)
AddMatch L (940, 1864)
AddMatch O (345, 1855)
AddMatch O (810, 2193)


{'D': [<Libs.Beholder.BeholderMatch at 0x7f69398b26b0>],
 'F': [<Libs.Beholder.BeholderMatch at 0x7f693993b250>],
 'L': [<Libs.Beholder.BeholderMatch at 0x7f69397580d0>],
 'O': [<Libs.Beholder.BeholderMatch at 0x7f6939759390>,
  <Libs.Beholder.BeholderMatch at 0x7f693975ac20>]}

In [ ]:
a.matchers

In [180]:
get_color_distance_fast(correct, gsColor(a.layers["image"], 1190, 1588))

175

In [ ]:
a.matchers["white_x"].show()

In [ ]:
a.layers["gray"].show()

In [ ]:
matches["T"][0].center

In [ ]:
matches["N"][0].center

shell input touchscreen swipe x1,y1, x2,y2

https://github.com/mattwilson1024/android-pattern-unlock/blob/master/unlock.sh

In [ ]:
generateMoves("FLEET", matches)

In [190]:
a.layers["image"].data[58][58]

array([74, 53, 41], dtype=uint8)

In [199]:
gsColor(a.layers["image"], 682, 64, [74, 57, 58])

True

In [196]:
gsColor(a.layers["image"], 1190, 1588)

array([58, 40, 33], dtype=uint8)